In [430]:
import layoutparser as lp
import cv2
from PIL import Image 
import pytesseract
from bs4 import BeautifulSoup

In [2]:
# Basic LP
model_config = 'lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config'
extra_config = "MODEL.ROI_HEADS.SCORE_THRESH_TEST"
label_map = {0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"}
model = lp.Detectron2LayoutModel(model_config, extra_config=[extra_config, 0.8], label_map=label_map)

The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [21]:

model_config = 'lp://HJDataset/faster_rcnn_R_50_FPN_3x/config'
extra_config = "MODEL.ROI_HEADS.SCORE_THRESH_TEST"
label_map = {1:"Page Frame", 2:"Row", 3:"Title Region", 4:"Text Region", 5:"Title", 6:"Subtitle", 7:"Other"}
model = lp.Detectron2LayoutModel(model_config, extra_config=[extra_config, 0.8], label_map=label_map)

config.yml?dl=1: 8.19kB [00:02, 2.79kB/s]
model_final.pth?dl=1: 330MB [01:03, 5.18MB/s]                                                                                                                                                              
The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [56]:
# Prima Layout
model_config = 'lp://PrimaLayout/mask_rcnn_R_50_FPN_3x/config'
extra_config = "MODEL.ROI_HEADS.SCORE_THRESH_TEST"
label_map = {1:"TextRegion", 2:"ImageRegion", 3:"TableRegion", 4:"MathsRegion", 5:"SeparatorRegion", 6:"OtherRegion"}
model = lp.Detectron2LayoutModel(model_config, extra_config=[extra_config, 0.8], label_map=label_map)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [57]:
image = cv2.imread("images/p-07.jpg")

In [58]:
layout = model.detect(image)

In [59]:
layout

Layout(_blocks=[TextBlock(block=Rectangle(x_1=794.5726928710938, y_1=3042.744140625, x_2=926.1603393554688, y_2=3084.89013671875), text=None, id=None, type=TextRegion, parent=None, next=None, score=0.999948263168335), TextBlock(block=Rectangle(x_1=517.1845092773438, y_1=3041.4267578125, x_2=676.8280639648438, y_2=3088.6689453125), text=None, id=None, type=TextRegion, parent=None, next=None, score=0.999893307685852), TextBlock(block=Rectangle(x_1=286.0674133300781, y_1=303.471435546875, x_2=2007.3709716796875, y_2=509.6767578125), text=None, id=None, type=TextRegion, parent=None, next=None, score=0.9963249564170837), TextBlock(block=Rectangle(x_1=522.8360595703125, y_1=1099.22021484375, x_2=733.0508422851562, y_2=1155.155029296875), text=None, id=None, type=TextRegion, parent=None, next=None, score=0.9957743287086487), TextBlock(block=Rectangle(x_1=1436.3953857421875, y_1=816.5255126953125, x_2=1797.8111572265625, y_2=888.7061157226562), text=None, id=None, type=TextRegion, parent=None,

In [60]:
#textual_blocks = lp.Layout([b for b in layout if b.type=='Text' or b.type == 'Title'])

In [61]:
img = lp.draw_box(image, layout,  box_width=10, box_alpha=0.5, show_element_type=True)

In [62]:
img.save('prima_layout3.jpg')

In [63]:
# Load image, grayscale, Gaussian blur, Otsu's threshold
image = cv2.imread('images/p-07.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (7,7), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Create rectangular structuring element and dilate
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
dilate = cv2.dilate(thresh, kernel, iterations=4)

# Find contours and draw rectangle
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)

# cv2.imshow('thresh', thresh)
# cv2.imshow('dilate', dilate)
# cv2.imshow('image', image)
# cv2.waitKey()

In [64]:
cv2.imwrite('cv.jpg', image)

True

In [65]:
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

2023-06-11 15:50:43.031943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 15:50:43.213751: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/dhruv/.local/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:246: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
2023-06-11 15:50:43.917962: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvin

In [587]:
model = ocr_predictor(pretrained=True)
# PDF
imgfile = 'images/p-10.jpg'
doc = DocumentFile.from_images(imgfile)
# Analyze
result = model(doc)

In [588]:
#result.show(doc)

In [589]:
# result

In [590]:
jsonresult = result.export()

In [591]:
height, width = jsonresult['pages'][0]['dimensions']

In [592]:
blockbboxes = []
for block in jsonresult['pages'][0]['blocks']:
    ((x1,y1),(x2,y2)) = block['geometry']
    x1 = int(x1 * width)
    x2 = int(x2 * width)
    y1 = int(y1 * height)
    y2 = int(y2 * height)
    bbox = ((x1,y1),(x2,y2))
    blockbboxes.append(bbox)

In [593]:
#blockbboxes

In [594]:
img = cv2.imread(imgfile)

In [595]:
for b in blockbboxes:
    cv2.rectangle(img, b[0], b[1], (255, 255, 25), 20)

In [596]:
cv2.imwrite(imgfile[:-4] + '_block_level.jpg', img)

True

In [597]:
## Line Level

In [598]:
linebboxes = []
for block in jsonresult['pages'][0]['blocks']:
    for lines in block['lines']:
        ((x1,y1),(x2,y2)) = lines['geometry']
        x1 = int(x1 * width)
        x2 = int(x2 * width)
        y1 = int(y1 * height)
        y2 = int(y2 * height)
        bbox = ((x1,y1),(x2,y2))
        linebboxes.append(bbox)

In [599]:
for b in linebboxes:
    cv2.rectangle(img, b[0], b[1], (25, 25, 255), -1)

In [600]:
cv2.imwrite(imgfile[:-4] + '_line_level.jpg', img)

True

In [601]:
img = cv2.imread(imgfile)

In [602]:
final_text = ''

In [603]:
img = cv2.imread(imgfile)

In [604]:
for bb in blockbboxes:
    cropped_image = img[bb[0][1]:bb[1][1], bb[0][0]:bb[1][0]]
    text = pytesseract.image_to_string(cropped_image, lang='Tamil')
    final_text = final_text + ' ' + text

In [605]:
print(final_text)

  FRUGOBSLSOSTN
Counter Wo: 2, 28/08/2021, 12:47

___—.

Aat:25.00 Cash Ut 120025

1
EE
Fros: TRADNRWFATTU 0 <&06707>
Frexx Hib | =EEa To
To: Hub’ |
Ll F0:Sarhiipi Hauer HCLIOOZD)
SN a

